In [43]:
import math
from itertools import *
from operator import itemgetter

In [2]:
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [3]:
def stirling_approx(n):
    return (n+.5)*math.log(n,2)-n+.5*math.log(2*math.pi,2)

In [32]:
def getBinaryStateTable(size, alphabet = [0,1]):
    if size == 1:
        return [[character] for character in alphabet]
        
    statetable_prev = getBinaryStateTable(size - 1)
    statetable_curr = [unfinished_state + [character] for character in alphabet for unfinished_state in statetable_prev]
    return statetable_curr

In [33]:
def log2P(statetable_dict):
    total = 0
    
    for key in statetable_dict.keys():
        outcomes = statetable_dict[key][0]
        num = 0; denom = 0; partial_sum = 0
        
        if (max(outcomes) > 50):
            num = sum([stirling_approx(o) for o in outcomes])
            denom = stirling_approx(sum(outcomes))
            partial_sum = num - denom
        else:
            num = reduce(lambda x,y:x*y, [math.factorial(o) for o in outcomes])
            denom = float(math.factorial(sum(outcomes)))
            partial_sum = math.log(num/denom, 2)
        total += partial_sum
    
    return total-math.log(len(statetable_dict.keys()), 2)*55

In [34]:
def bayesian_network(features_list, desired_features_index, sample_outcomes):    
    statetable = [tuple(state) for state in getBinaryStateTable(len(desired_features_index))]
    statetable_dict = {}
    for state in statetable:
        statetable_dict[state] = [[0]*len(sample_outcomes), [0]*len(sample_outcomes)]
    
    # Counts the total number of samples of being in a given cell state for each feature state
    for sample in features_list:
        state_sample = tuple([sample[i] for i in desired_features_index])
        N = sample[6] # cell state
        statetable_dict[state_sample][0][N] += 1;
    
    # Calculates the probabilities
    for key in statetable_dict.keys():
        tot = float(sum(statetable_dict[key][0]))
        for i in range(len(sample_outcomes)):
            statetable_dict[key][1][i] = statetable_dict[key][0][i]/tot
    
    loglikelihood = log2P(statetable_dict)
    
    return desired_features_index, loglikelihood

In [60]:
def exhaustive_model_search(features_list, parameters):
    allmodels = list(powerset(parameters))
    bayesian_models = []
    for model in allmodels:
        if list(model) != []:
            bayesian_models.append(bayesian_network(features_list, list(model), [0,1]))
    return sorted(bayesian_models, key=lambda x: x[1], reverse=True)

In [62]:
def main():
    iPSC1 = open("ipsc1.dat", "r")
    iPSC1_list = iPSC1.readlines()
    iPSC1_list = [map(int, line.rstrip().split(" ")) for line in iPSC1_list]
    bayesian_models = exhaustive_model_search(iPSC1_list, [0,1,2,3,4,5])
    
    for model in bayesian_models:
        print(model)
main()

([0, 1, 2], -1269.1600705391697)
([0, 1, 2, 3, 4, 5], -1281.070973434198)
([0, 1, 2, 3], -1292.3228055149268)
([0, 1, 2, 4], -1295.133446589482)
([0, 1, 2, 5], -1297.6042382748524)
([0, 1, 2, 3, 4], -1299.0826667818048)
([0, 1, 2, 3, 5], -1303.15706101261)
([0, 1, 2, 4, 5], -1306.226910256871)
([0, 1], -1317.7449421656356)
([0, 2], -1332.1402494750942)
([0, 1, 3], -1351.9188098167697)
([0, 1, 4], -1354.5499262311757)
([0, 1, 5], -1356.6243713414572)
([0, 2, 3], -1368.4710933789202)
([0, 2, 4], -1369.5804074137727)
([0, 2, 5], -1370.7725007895954)
([0], -1373.521677515958)
([0, 1, 3, 4], -1375.5114267390486)
([0, 1, 3, 5], -1379.2309319426922)
([0, 1, 4, 5], -1381.6959451083478)
([0, 1, 3, 4, 5], -1382.6049263940872)
([0, 2, 3, 4], -1393.1164955909617)
([0, 2, 3, 5], -1395.2031449601418)
([0, 2, 4, 5], -1397.7759411660165)
([0, 2, 3, 4, 5], -1399.8691124898792)
([0, 4], -1417.1305888043403)
([0, 3], -1417.5623075636486)
([0, 5], -1418.7632011121445)
([0, 3, 4], -1453.4549415390816)
([0,